# 🧹 Data Cleaning with Pandas - Complete Tutorial

This tutorial will teach you how to clean **dirty data** using **pandas**. We'll work with a real-world-like messy dataset and fix common data quality issues.

## What You'll Learn:
1. Identifying data quality issues
2. Handling missing values
3. Removing duplicates
4. Fixing inconsistent formatting (casing, whitespace)
5. Handling outliers and invalid values
6. Standardizing date formats
7. Data type conversions

---

## Step 1: Import Libraries and Load Data

In [2]:
import pandas as pd
import numpy as np

# Load the dirty dataset
df = pd.read_csv('dirty_employees.csv')

# Display basic info
print("Shape of dataset:", df.shape)
print("\nFirst 10 rows:")
df.head(10)

Shape of dataset: (50, 10)

First 10 rows:


,employee_id,first_name,last_name,email,department,salary,hire_date,age,city,phone_number
0,1,John,Smith,john.smith@company.com,Sales,55000.0,2020-01-15,28,New York,555-1234
1,2,Jane,Doe,jane.doe@company.com,Marketing,62000.0,2019-03-22,32,Los Angeles,555-5678
2,3,MICHAEL,Johnson,michael.johnson@company.com,IT,75000.0,2018-07-10,35,Chicago,555-9012
3,4,emily,Brown,emily.brown@company.com,HR,48000.0,2021-05-18,26,Houston,555-3456
4,5,David,Williams,david.williams@company.com,Sales,58000.0,2020-08-25,30,Phoenix,555-7890
5,6,Sarah,Miller,NaN,Finance,67000.0,2019-11-03,34,Philadelphia,(555) 2345
6,7,James,Davis,james.davis@company.com,IT,NaN,2017-02-14,38,San Antonio,555.6789
7,8,Jennifer,Garcia,jennifer.garcia@company.com,Marketing,54000.0,2020-04-30,29,San Diego,555-0123
8,9,Robert,Martinez,robert.martinez@company.com,Sales,56000.0,2019-09-12,31,Dallas,555-4567
9,10,Lisa,Anderson,lisa.anderson@company.com,HR,51000.0,2021-01-20,27,San Jose,5558901


---
## Step 2: Explore the Data (Find the Problems!)

Before cleaning, we need to understand what's wrong with the data.

In [3]:
# Check data types
print("Data Types:")
print(df.dtypes)
print("\n" + "="*50)

Data Types:
employee_id       int64
first_name       object
last_name        object
email            object
department       object
salary          float64
hire_date        object
age               int64
city             object
phone_number     object
dtype: object



In [4]:
# Check for missing values
print("Missing Values Count:")
print(df.isnull().sum())
print("\nTotal Missing Values:", df.isnull().sum().sum())

Missing Values Count:
employee_id     0
first_name      1
last_name       0
email           1
department      0
salary          3
hire_date       0
age             0
city            0
phone_number    0
dtype: int64

Total Missing Values: 5


In [ ]:
# Check for duplicates
print("Number of duplicate rows:", df.duplicated().sum())
print("\nDuplicate rows:")
df[df.duplicated(keep=False)]

In [ ]:
# Check unique values in categorical columns
print("Unique departments (notice the inconsistencies):")
print(df['department'].unique())

In [ ]:
# Look at salary statistics (notice outliers)
print("Salary Statistics:")
print(df['salary'].describe())
print("\nUnique salary values that look suspicious:")
print(df['salary'].unique())

---
## 📋 Problems We Found:

1. **Missing values**: Empty cells, 'NULL', 'N/A' in various columns
2. **Duplicate rows**: Row 1 and 11 are identical
3. **Inconsistent casing**: 'Sales', 'sales', 'MICHAEL', 'emily'
4. **Whitespace issues**: ' Sales ', '   HR   ', ' Fort Worth'
5. **Invalid values**: Negative salary (-5000), negative age (-25)
6. **Outliers**: Salary of 650000 (typo), salary of 150 (too low)
7. **Inconsistent date formats**: '2020-01-15' vs '15-03-2021'
8. **Inconsistent phone formats**: '555-1234', '(555) 2345', '555.6789', '5558901'

---

## Step 3: Handle Missing Values

### 3.1 Replace text placeholders with NaN

In [ ]:
# Make a copy to preserve original
df_clean = df.copy()

# Replace common missing value representations with NaN
df_clean = df_clean.replace(['NULL', 'N/A', 'n/a', 'NA', '', ' '], np.nan)

print("Missing values after replacement:")
print(df_clean.isnull().sum())

### 3.2 Handle Missing Values - Different Strategies

In [ ]:
# STRATEGY 1: Drop rows with missing values (use when you have lots of data)
# df_clean = df_clean.dropna()

# STRATEGY 2: Fill with specific values
# For salary: fill with median (more robust than mean for outliers)
salary_median = df_clean['salary'].median()
print(f"Median salary: {salary_median}")

df_clean['salary'] = df_clean['salary'].fillna(salary_median)

# For first_name: fill with 'Unknown'
df_clean['first_name'] = df_clean['first_name'].fillna('Unknown')

# For email: we could construct it from name, or leave as 'no.email@company.com'
df_clean['email'] = df_clean['email'].fillna('no.email@company.com')

print("\nMissing values after filling:")
print(df_clean.isnull().sum())

---
## Step 4: Remove Duplicates

In [ ]:
print(f"Rows before removing duplicates: {len(df_clean)}")

# Remove duplicate rows (keep first occurrence)
df_clean = df_clean.drop_duplicates(keep='first')

print(f"Rows after removing duplicates: {len(df_clean)}")

# You can also check for duplicates based on specific columns
# df_clean = df_clean.drop_duplicates(subset=['email'], keep='first')

---
## Step 5: Fix Inconsistent Casing and Whitespace

In [ ]:
# Fix department: strip whitespace and title case
df_clean['department'] = df_clean['department'].str.strip().str.title()

print("Unique departments after cleaning:")
print(df_clean['department'].unique())

In [ ]:
# Fix names: strip whitespace and title case
df_clean['first_name'] = df_clean['first_name'].str.strip().str.title()
df_clean['last_name'] = df_clean['last_name'].str.strip().str.title()

# Fix city: strip whitespace and title case
df_clean['city'] = df_clean['city'].str.strip().str.title()

# Fix email: lowercase
df_clean['email'] = df_clean['email'].str.strip().str.lower()

print("Sample of cleaned names:")
df_clean[['first_name', 'last_name', 'email', 'city']].head(10)

---
## Step 6: Handle Invalid Values and Outliers

In [ ]:
# Find rows with invalid salary (negative values)
print("Rows with negative salary:")
print(df_clean[df_clean['salary'] < 0])

In [ ]:
# Convert salary to numeric first (in case there are strings)
df_clean['salary'] = pd.to_numeric(df_clean['salary'], errors='coerce')

# Fix negative values: replace with median
salary_median = df_clean[df_clean['salary'] > 0]['salary'].median()
df_clean.loc[df_clean['salary'] <= 0, 'salary'] = salary_median

print(f"After fixing negative salaries - Min: {df_clean['salary'].min()}, Max: {df_clean['salary'].max()}")

In [ ]:
# Handle outliers using IQR method
Q1 = df_clean['salary'].quantile(0.25)
Q3 = df_clean['salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"IQR: {IQR}")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")

# Find outliers
outliers = df_clean[(df_clean['salary'] < lower_bound) | (df_clean['salary'] > upper_bound)]
print(f"\nNumber of outliers: {len(outliers)}")
print(outliers[['employee_id', 'first_name', 'salary']])

In [ ]:
# Option 1: Cap outliers at bounds
df_clean['salary'] = df_clean['salary'].clip(lower=lower_bound, upper=upper_bound)

# Option 2: Replace with median (uncomment to use)
# df_clean.loc[(df_clean['salary'] < lower_bound) | (df_clean['salary'] > upper_bound), 'salary'] = salary_median

print(f"After handling outliers - Min: {df_clean['salary'].min()}, Max: {df_clean['salary'].max()}")

In [ ]:
# Fix negative age values
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
df_clean.loc[df_clean['age'] < 0, 'age'] = df_clean['age'].abs()  # Take absolute value

print(f"Age range: {df_clean['age'].min()} to {df_clean['age'].max()}")

---
## Step 7: Standardize Date Formats

In [ ]:
# Check current date values
print("Sample hire_date values:")
print(df_clean['hire_date'].head(20).values)

In [ ]:
# Convert to datetime with multiple format inference
# The 'dayfirst' parameter helps with ambiguous dates
df_clean['hire_date'] = pd.to_datetime(df_clean['hire_date'], dayfirst=False, errors='coerce')

# For dates that might be in different format (DD-MM-YYYY)
# We need a custom function to handle mixed formats
def parse_mixed_dates(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        # Try standard format first (YYYY-MM-DD)
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except:
        try:
            # Try DD-MM-YYYY format
            return pd.to_datetime(date_str, format='%d-%m-%Y')
        except:
            return pd.NaT

# Reload and apply custom parsing
df_temp = pd.read_csv('dirty_employees.csv')
df_clean['hire_date'] = df_temp['hire_date'].apply(parse_mixed_dates)

print("Hire dates after conversion:")
print(df_clean['hire_date'].head(20))

---
## Step 8: Standardize Phone Numbers

In [ ]:
# Check current phone formats
print("Current phone number formats:")
print(df_clean['phone_number'].unique())

In [ ]:
import re

def standardize_phone(phone):
    if pd.isna(phone):
        return np.nan
    # Remove all non-numeric characters
    digits = re.sub(r'\D', '', str(phone))
    # Format as XXX-XXXX (assuming 7 digits)
    if len(digits) == 7:
        return f"{digits[:3]}-{digits[3:]}"
    # Format as (XXX) XXX-XXXX for 10 digits
    elif len(digits) == 10:
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
    return digits

df_clean['phone_number'] = df_clean['phone_number'].apply(standardize_phone)

print("Standardized phone numbers:")
print(df_clean['phone_number'].head(15))

---
## Step 9: Reset Index and Final Check

In [ ]:
# Reset index after removing rows
df_clean = df_clean.reset_index(drop=True)

# Final overview
print("=" * 50)
print("FINAL CLEANED DATASET SUMMARY")
print("=" * 50)
print(f"\nShape: {df_clean.shape}")
print(f"\nMissing values:\n{df_clean.isnull().sum()}")
print(f"\nDuplicates: {df_clean.duplicated().sum()}")
print(f"\nData types:\n{df_clean.dtypes}")

In [ ]:
# View the cleaned data
df_clean.head(20)

In [ ]:
# Verify salary statistics are reasonable now
print("Cleaned Salary Statistics:")
print(df_clean['salary'].describe())

---
## Step 10: Save the Cleaned Data

In [ ]:
# Save to a new CSV file
df_clean.to_csv('cleaned_employees.csv', index=False)
print("✅ Cleaned data saved to 'cleaned_employees.csv'")

---
# 📚 Quick Reference Cheatsheet

## Common Pandas Cleaning Functions

| Issue | Solution |
|-------|----------|
| Missing values | `df.isnull().sum()`, `df.fillna(value)`, `df.dropna()` |
| Duplicates | `df.duplicated()`, `df.drop_duplicates()` |
| Whitespace | `df['col'].str.strip()` |
| Case issues | `df['col'].str.lower()`, `.upper()`, `.title()` |
| Replace values | `df.replace(old, new)` |
| Convert types | `df['col'].astype(type)`, `pd.to_numeric()`, `pd.to_datetime()` |
| Outliers | `df['col'].clip(lower, upper)` |
| Filter data | `df[df['col'] > value]` |
| Apply function | `df['col'].apply(func)` |

---

## 🎯 Practice Exercises

Try these on your own:

1. Create a new column `full_name` by combining first_name and last_name
2. Calculate the average salary by department
3. Find employees hired before 2019
4. Create age groups (20-25, 26-30, 31-35, 36-40)
5. Find the department with the highest average salary

In [ ]:
# Practice Exercise 1: Create full_name
df_clean['full_name'] = df_clean['first_name'] + ' ' + df_clean['last_name']
df_clean['full_name'].head()

In [ ]:
# Practice Exercise 2: Average salary by department
df_clean.groupby('department')['salary'].mean().sort_values(ascending=False)

In [ ]:
# Practice Exercise 3: Employees hired before 2019
df_clean[df_clean['hire_date'] < '2019-01-01'][['full_name', 'department', 'hire_date']]

In [ ]:
# Practice Exercise 4: Create age groups
bins = [20, 25, 30, 35, 40, 100]
labels = ['20-25', '26-30', '31-35', '36-40', '40+']
df_clean['age_group'] = pd.cut(df_clean['age'], bins=bins, labels=labels)
df_clean['age_group'].value_counts()

In [ ]:
# Practice Exercise 5: Department with highest average salary
print("Department with highest average salary:")
print(df_clean.groupby('department')['salary'].mean().idxmax())